In [18]:
import os
from tqdm import tqdm
os.chdir('/home/jangting/2021_summer/AISE/fooling/LeNet5-MNIST-PyTorch')

from model import Model
import numpy as np
import torch
from torchvision.datasets import mnist
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

In [21]:
def main():
    batch_size = 256
    epoch = 100
    
    train_dataset = mnist.MNIST(root='./train', train=True, transform=ToTensor())
    test_dataset = mnist.MNIST(root='./test', train=False, transform=ToTensor())
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    
    model = Model()
    sgd = SGD(model.parameters(), lr=1e-1)
    cost = CrossEntropyLoss()
    
    previous_correct = 0

    print(len(train_loader))

    for _epoch in range(epoch):
        for idx, (train_x, train_label) in enumerate(train_loader):
            label_np = np.zeros((train_label.shape[0], 10))
            sgd.zero_grad()
            predict_y = model(train_x.float())
            loss = cost(predict_y, train_label.long())
            #if idx % 50 == 0:
            #    print('idx: {}, loss: {}'.format(idx, loss.sum().item()))
            loss.backward()
            sgd.step()

        correct = 0
        _sum = 0

        for idx, (test_x, test_label) in enumerate(test_loader):
            predict_y = model(test_x.float()).detach()
            predict_ys = np.argmax(predict_y, axis=-1)
            label_np = test_label.numpy()
            _ = predict_ys == test_label
            correct += np.sum(_.numpy(), axis=-1)
            _sum += _.shape[0]

        if previous_correct < correct: 
            print('epoch : {},\taccuracy: {:.2f}%'.format(_epoch ,correct / _sum * 100))
            torch.save(model, 'models/mnist_{:.4f}.pkl'.format(correct / _sum))
            previous_correct = correct

if __name__ == '__main__':
    main()

235
epoch : 0,	accuracy: 53.29%
epoch : 1,	accuracy: 80.86%
epoch : 2,	accuracy: 85.64%
epoch : 3,	accuracy: 86.27%
epoch : 4,	accuracy: 87.19%
epoch : 5,	accuracy: 87.68%
epoch : 7,	accuracy: 87.83%
epoch : 8,	accuracy: 87.99%
epoch : 9,	accuracy: 88.07%
epoch : 10,	accuracy: 88.13%
epoch : 11,	accuracy: 88.20%
epoch : 14,	accuracy: 88.23%
epoch : 19,	accuracy: 88.27%
epoch : 36,	accuracy: 88.40%
epoch : 39,	accuracy: 88.61%
